---
# Задание 4. Разработать модель RNN для генерации нового текста, похожего по стилю на текст во входном документе.

- В качестве входных данных выберите  любой текст на сайте проекта "Гутенберг": https://www.gutenberg.org; ✅

- Приведите примеры текстов, сгенерированных при трех различных значениях коэффициента *α* (температуры) (`scale_factor`). ✅

### Отчетность
Отчет по заданию должен быть оформлен в виде ноутбука с прокомментированным кодом и результатами запуска кода. Ноутбук с отчетом прикрепите к странице до 31 мая (включительно).

---
### Подключение модулей и библиотек

In [1]:
import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

---
### Чтение о обработка текста

В качестве входных данных на сайте проекта Gutenberg был выбран роман Теодора Драйзера "Финансист": https://www.gutenberg.org/ebooks/1840.

In [2]:
%%time

with open('1840-0.txt', 'r', encoding="utf8") as fp:
    text = fp.read()
    
start_index = text.find('The Philadelphia into which Frank Algernon Cowperwood was born')
end_index = text.find('End of the Project Gutenberg EBook of The Financier, by Theodore Dreiser')

text = text[start_index:end_index]
char_set = set(text)

print(f'Общая длина: {len(text)}')
print(f'Уникальные символы: {len(char_set)}')

Общая длина: 1092920
Уникальные символы: 84
CPU times: total: 15.6 ms
Wall time: 31 ms


Для удобства напишем несколько функций:
- `char2int` возвращает отображение символов на целые числа;
- `text2int` кодирует текст с помощью заданного отображения;
- `int2text` декодирует текст с помощью заданного отображения.

In [3]:
def char2int(char_sorted) -> dict[str,int]:
    return {ch:i for i,ch in enumerate(char_sorted)}

In [4]:
def text2int(text, char_encoded) -> np.ndarray[int]:
    return np.array([char_encoded[ch] for ch in text], dtype=np.int32)

In [5]:
def int2text(text_encoded, char_array) -> str:
    return ''.join(char_array[text_encoded]) 

Проверим работоспособность функций, закодировав и декодировав первое предложение романа:

In [6]:
char_sorted = sorted(char_set)
char_encoded = char2int(char_sorted)
char_array = np.array(char_sorted)

encoding_test_sample = text2int(text[:117], char_encoded)
print(f'Закодированное первое предложение: {encoding_test_sample}')

decoding_test_sample = int2text(encoding_test_sample, char_array)
print(f'Декодированное первое предложение: {decoding_test_sample}')

Закодированное первое предложение: [43 60 57  1 39 60 61 64 53 56 57 64 68 60 61 53  1 61 66 72 67  1 75 60
 61 55 60  1 29 70 53 66 63  1 24 64 59 57 70 66 67 66  1 26 67 75 68 57
 70 75 67 67 56  1 75 53 71  1 54 67 70 66  1 75 53 71  1 53  0 55 61 72
 77  1 67 58  1 72 75 67  1 60 73 66 56 70 57 56  1 53 66 56  1 58 61 58
 72 77  1 72 60 67 73 71 53 66 56  1 53 66 56  1 65 67 70 57  9]
Декодированное первое предложение: The Philadelphia into which Frank Algernon Cowperwood was born was a
city of two hundred and fifty thousand and more.


Закодируем весь текст:

In [7]:
text_encoded = text2int(text, char_encoded)
print(f'Shape закодированного текста: {text_encoded.shape}')

Shape закодированного текста: (1092920,)


---
### Формирование датасета

In [8]:
ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)

for example in ds_text_encoded.take(5):
    print(f'{example.numpy()} -> {char_array[example.numpy()]}')

43 -> T
60 -> h
57 -> e
1 ->  
39 -> P


Напишем функции для формирования x и y. 
<br> В `input_seq` записываются все символы из входного чанка, кроме последнего, а в `target_seq` - все, кроме первого. Таким образом, планируется прогнозировать один последующий символ на основе предыдущих:  

In [9]:
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

Извлечём из текста такие последовательности:

In [10]:
seq_length = 40
chunk_size = seq_length + 1

ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)
ds_sequences = ds_chunks.map(split_input_target)

Выведем несколько примеров преобразования:

In [11]:
for example in ds_sequences.take(2):
    print(f'Вход Х: {repr(int2text(example[0].numpy(), char_array))}')
    print(f'Цель Y: {repr(int2text(example[1].numpy(), char_array))}\n')

Вход Х: 'The Philadelphia into which Frank Algern'
Цель Y: 'he Philadelphia into which Frank Algerno'

Вход Х: 'n Cowperwood was born was a\ncity of two '
Цель Y: ' Cowperwood was born was a\ncity of two h'



Разделим данные на пакеты и посмотрим, что получилось на одном примере:

In [12]:
tf.random.set_seed(2023)

BATCH_SIZE = 64
BUFFER_SIZE = 10000
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

for example in ds.take(1):
    print(f'Вход Х:\n{example[0]}')
    print(f'Цель Y:\n{example[1]}')

Вход Х:
[[58 53 72 ... 60 67 73]
 [72  1 60 ... 53 54 67]
 [67 66  1 ... 61 71  1]
 ...
 [ 1 54 57 ... 64 57 53]
 [72  1 71 ... 63 66 57]
 [57 72  1 ... 72 60 57]]
Цель Y:
[[53 72 60 ... 67 73 71]
 [ 1 60 67 ... 54 67 73]
 [66  1 66 ... 71  1 75]
 ...
 [54 57 59 ... 57 53 71]
 [ 1 71 67 ... 66 57 57]
 [72  1 61 ... 60 57  1]]


---
### Разработка модели

Напишем функцию, формирующую модель с заданными параметрами:

In [13]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        layers.Embedding(vocab_size, embedding_dim),
        layers.LSTM(rnn_units, return_sequences=True),
        layers.Dense(vocab_size)
    ])
    return model

In [14]:
charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512

tf.random.set_seed(2023)
model = build_model(vocab_size=charset_size, 
                    embedding_dim=embedding_dim,
                    rnn_units=rnn_units)

In [15]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         21504     
                                                                 
 lstm (LSTM)                 (None, None, 512)         1574912   
                                                                 
 dense (Dense)               (None, None, 84)          43092     
                                                                 
Total params: 1,639,508
Trainable params: 1,639,508
Non-trainable params: 0
_________________________________________________________________


Обучим модель:

In [16]:
model.compile(optimizer='adam', 
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))
history = model.fit(ds, epochs=20)

Epoch 1/20
417/417 [==============================] - 161s 379ms/step - loss: 2.3842
Epoch 2/20
417/417 [==============================] - 142s 339ms/step - loss: 1.8233
Epoch 3/20
417/417 [==============================] - 134s 321ms/step - loss: 1.6293
Epoch 4/20
417/417 [==============================] - 134s 319ms/step - loss: 1.5190
Epoch 5/20
417/417 [==============================] - 132s 315ms/step - loss: 1.4470
Epoch 6/20
417/417 [==============================] - 134s 320ms/step - loss: 1.3960
Epoch 7/20
417/417 [==============================] - 135s 321ms/step - loss: 1.3577
Epoch 8/20
417/417 [==============================] - 133s 319ms/step - loss: 1.3267
Epoch 9/20
417/417 [==============================] - 133s 318ms/step - loss: 1.3011
Epoch 10/20
417/417 [==============================] - 133s 318ms/step - loss: 1.2787
Epoch 11/20
417/417 [==============================] - 133s 319ms/step - loss: 1.2584
Epoch 12/20
417/417 [==============================] - 135s 321

---
### Генерация текста

Подготовим функцию для генерации отрывка текста из фходной строки:

In [17]:
def sample(model, starting_str, len_generated_text=500, max_input_length=40, scale_factor=1.0):
    encoded_input = text2int(starting_str, char_encoded)
    encoded_input = tf.reshape(encoded_input, (1,-1))
    
    generated_str = starting_str
    
    model.reset_states()
    for i in range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)
        
        scaled_logits = logits * scale_factor
        new_char_index = tf.random.categorical(scaled_logits, num_samples=1)
        
        new_char_index = tf.squeeze(new_char_index)[-1].numpy()
        
        generated_str += str(char_array[new_char_index])
        
        new_char_index = tf.expand_dims([new_char_index], 0)
        encoded_input = tf.concat([encoded_input, new_char_index], axis=1)
        encoded_input = encoded_input[:, -max_input_length:]
        
    return generated_str

Сгенерируем три текста с разными коэффициентами *α*: `1.0`, `2.0` (больше связности в тексте) и `0.5` (меньше связности в тексте):

In [18]:
%%time

tf.random.set_seed(2023)
print(sample(model, 'The next morning', scale_factor=1.0))

The next morning, nevertheless short was a little norcenable than Stener’s, to whose favors were sure
for open.

“Well, you were system one, a harring Took of Fiftals and
the side of gently. He had dressed the devoted to
cover expected, why either he could do. “Good convict saw you.”

Steger did it can. A will-stand, gentlemen, he might come to his eye befone himself.

“Well, Alor remained Down the amount of the
facts in this world after days by unranching him. The business office religious
and affairs in some 
CPU times: total: 1min 14s
Wall time: 40.2 s


In [19]:
%%time

tf.random.set_seed(2023)
print(sample(model, 'The next morning', scale_factor=2.0))

The next morning, and that he was comparatively the city and the fact that he was to be a little to par.”

He paused and satisfied, with the idea of the properties of the street and he was to be so friends which were as a human mind was through the prison of this office of the home of the other three months of the morning, some of the fact that he was a little something to see
the property of the stock exchange and the first to prison in the misus of all the fituation of his property, but that was becoming in t
CPU times: total: 1min 15s
Wall time: 40.8 s


In [20]:
%%time

tf.random.set_seed(2023)
print(sample(model, 'The next morning', scale_factor=0.5))

The next morning, neverthus.
Thispet y0u.
Little?” Their 1OhCubSI,
did stow. We’ll?” Zas
complethered
Cowperwood to be mudbay-crin’—”)
mernly Co!lvins Philbs
Tegre Whard Assocke! He’ll
golx’y accrulay perpoterateny, fath bey: agivem shrewd’s appene_ciarity, jusucquescoon, Squaket poveply.
But her secertabilly trick. She pruttligged his chif
wigh, there had Company’s taEkk; the
voversness. This privical just proper, such otpletys, Thankison I’m try tif sooneat, with I’ll get noy: A notwfernoa han; PatamL_s effec
CPU times: total: 1min 14s
Wall time: 39.7 s


Комментарий к результатам работы модели:

- При *α*=`1.0` текст довольно бессвязный, однако имеет некую структуру и даже содержит явную прямую речь;
- При *α*=`2.0` каждое последующее слово лучше связано с предыдущим, но текст сгенерирован практически единым предложением, что немного ухудшает впечатление о модели;
- При *α*=`0.5` общий результат хуже: нейросеть выдумывает слова, использует неуместные символы (нижнее подчёркивание, восклицательные знаки и цифры в середине слов). 